<a href="https://colab.research.google.com/github/silent-ai/chessboard/blob/main/nbs/chessboard_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
! pip install fastai --upgrade > /dev/null

     |████████████████████████████████| 194kB 15.4MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 


In [2]:
from fastai.vision.all import *

## Download data from github

In [3]:
from zipfile import ZipFile as unzip
path = Path('data')

#with unzip(download_data('https://tinyurl.com/silentai'), 'r') as ref:
 #   ref.extractall(path)

In [4]:
with unzip(download_data('https://github.com/silent-ai/chessboard/blob/main/data/small_amazon.zip?raw=true'), 'r') as ref:
    ref.extractall(path)

## Get annotations

In [5]:
import json
with open(path/'annotations'/'instances_default.json') as f:
  codes = json.load(f)

In [6]:
# Join the images to their label by id
img = {img['id']: img['file_name'] for img in codes['images']}
ann = {img['id']: img['segmentation'][0] for img in codes['annotations']}
labels = {img[id]: list(zip(a[::2], a[1::2])) for id, a in ann.items()}

# Grab the first example
first(labels.items())

('PXL_20201202_184115342_RESIZE.jpg',
 [(1031.89, 100.85), (685.6, 531.25), (925.34, 1086.42), (1424.43, 734.53)])

In [7]:
def get_label(f): return tensor(labels[f.name])

In [22]:
get_label(path/'images'/'PXL_20201202_183829674_RESIZE.jpg')

img_files = get_image_files(path)
#def img2pose(x): return Path(f'{str(x)[:-7]}pose.txt')
#img2pose(img_files[0])
img_files[0]
Image.open(img_files[0])

<bound method Image.show of <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2016x1134 at 0x7F70BDD2B438>>

## Datablock API

In [9]:
biwi = DataBlock(
    blocks=(ImageBlock, PointBlock),
    get_items=get_image_files,
    get_y=get_label,
    #item_tfms=Resize(320),
    batch_tfms=[Normalize.from_stats(*imagenet_stats)]
)


In [16]:
#imgs = get_image_files(path/'content/chessboard/data/chessboard_images')
#def overwriter(fname): return 'chessboard'
#lambda is like def but anonomous 
chess = DataBlock(
    #TensorCategory(0) is type for Category
    blocks=(ImageBlock, PointBlock),
    get_items=get_image_files,
    
    #get image file output passed into overwriter call
    #ML: x is input, y is label
    get_y= get_label,
    
    #specify validation set
    
    #splitter=RandomSplitter(),

    #specify item transformations
    item_tfms=[Normalize.from_stats(*imagenet_stats)]
    
)
chessboard_dls = chess.dataloaders(path)
#chessboard_dls.show_batch()

chesslearn = cnn_learner(chessboard_dls, resnet34)
#n out is number of categories out 
#res34 is arch which has been pretrained already so has classifications like man, dog, frog

chesslearn.lr_find()


Could not do one pass in your dataloader, there is something wrong in it


ZeroDivisionError: ignored

In [ ]:
dls = biwi.dataloaders(path/'content', bs=4)
dls.show_batch(max_n=9, figsize=(8,6))

## Train

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(-1,1))

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(3, 5e-2)

In [ ]:
learn.show_results()